In [64]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup as bs
import os
import re
from api_keys import *

In [62]:
# get artist object from genius api
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response

def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs

request_song_url('luke bryan', 20)

Found 20 songs by luke bryan


['https://genius.com/Luke-bryan-most-people-are-good-lyrics',
 'https://genius.com/Luke-bryan-rain-is-a-good-thing-lyrics',
 'https://genius.com/Luke-bryan-thats-my-kind-of-night-lyrics',
 'https://genius.com/Luke-bryan-what-she-wants-tonight-lyrics',
 'https://genius.com/Luke-bryan-play-it-again-lyrics',
 'https://genius.com/Luke-bryan-light-it-up-lyrics',
 'https://genius.com/Luke-bryan-country-girl-shake-it-for-me-lyrics',
 'https://genius.com/Luke-bryan-strip-it-down-lyrics',
 'https://genius.com/Luke-bryan-drunk-on-you-lyrics',
 'https://genius.com/Luke-bryan-huntin-fishin-and-lovin-every-day-lyrics',
 'https://genius.com/Luke-bryan-home-alone-tonight-lyrics',
 'https://genius.com/Luke-bryan-fast-lyrics',
 'https://genius.com/Luke-bryan-drink-a-beer-lyrics',
 'https://genius.com/Luke-bryan-build-me-a-daddy-lyrics',
 'https://genius.com/Luke-bryan-knockin-boots-lyrics',
 'https://genius.com/Luke-bryan-kick-the-dust-up-lyrics',
 'https://genius.com/Luke-bryan-what-makes-you-country-

In [51]:
def get_attrs(html):
  script = html.find('script', string=re.compile(r'var targeting_list ='))
  if script:
    script_string = script.string
    match = re.search(r'var targeting_list = (\[.*?\]);', script_string)
    if match:
      json_string = match.group(1)
      targeting_list = json.loads(json_string) # dict
    else:
      print('no targeting_list')
  else:
    print('script not found')
  targeting_list = {item['name']: item['values'] for item in targeting_list}
  return targeting_list

In [67]:
def scrape(url):
    page = requests.get(url)
    html = bs(page.text, 'html.parser')

    targeting_list = get_attrs(html)
    title = targeting_list['song_title'][0]
    tag = targeting_list['primary_tag'][0]
    artist = targeting_list['artist_name'][0]
    year = targeting_list['release_year'][0]
    views = targeting_list['pageviews'][0]

    lyrics = html.find(attrs={"data-lyrics-container": "true"}).get_text(' ')
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    return title, tag, artist, year, views, lyrics

scrape('https://genius.com/Luke-bryan-sunrise-sunburn-sunset-lyrics')

('Sunrise Sunburn Sunset',
 'country',
 'Luke Bryan',
 '2017',
 '21347',
 " Your daddy had a lakehouse He had me paint it that summer sophomore year Said I could stay for free A couple buddies, a couple weeks You rode in in a little red Honda And headed straight for the pier Tank top and cutoff jeans A little PacSun underneath Who knew it was 'bout to be  Sunrise, sunburn, sunset, repeat Moonlight, all night, crashing into me Nothing will ever be easy as you and me Tangled up with nowhere to be Just sunrise, sunburn, sunset, repeat Sunrise, sunburn, sunset, repeat, ayy  I turned those shutters the color of your eyes 'Til the day turned into night And took turns with a jar Turn that boat dock into a bar And then we passed the guitar around the fire to each other And watched all of our friends pair off with one another But we felt like the stars we were under And everybody knew that that was our summer")

In [69]:
df = pd.DataFrame(columns=['title', 'tag', 'artist', 'year', 'views', 'lyrics'])
titles = []
tags = []
artists = []
years = []
views = []
lyrics = []
for url in request_song_url('luke bryan', 20):
    title, tag, artist, year, view, lyric = scrape(url)
    titles.append(title)
    tags.append(tag)
    artists.append(artist)
    years.append(year)
    views.append(view)
    lyrics.append(lyric)
df['title'] = titles
df['tag'] = tags
df['artist'] = artists
df['year'] = years
df['views'] = views
df['lyrics'] = lyrics
df.head()

Found 20 songs by luke bryan


,title,tag,artist,year,views,lyrics
0,Most People Are Good,country,Luke Bryan,2017,125629,I believe kids oughta stay kids as long as th...
1,Rain is a Good Thing,country,Luke Bryan,2009,89234,My daddy spent his life lookin' up at the sky...
2,Thats My Kind of Night,country,Luke Bryan,2013,75796,"I got that real good, feel good stuff Up unde..."
3,What She Wants Tonight,country,Luke Bryan,2019,79121,"She walks up, velvet rope unhooks She snaps h..."
4,Play It Again,country,Luke Bryan,2014,67162,She was sittin' all alone over on the tailgat...
